In [7]:
"Check data"
import numpy as np 

data = np.load("C:\\Users\\msi\\Desktop\\Constanze\\Docs\\DATA\\PREPROCESSED\\S01\\data.npy")
print(data.shape)
nan_count = np.isnan(data).sum()

# Print the result
print(f"Number of NaN values in the array: {nan_count}")

(461, 64, 7196)
Number of NaN values in the array: 0


In [3]:
from pathlib import Path
import numpy as np

# Define data directory as a Path object
data_dir = Path("C:\\Users\\msi\\Desktop\\Constanze\\Docs\\DATA\\PREPROCESSED\\")

# Find all sessions matching the pattern "S*"
trials = sorted(data_dir.glob("S*"))

print(f"Found sessions: {len(trials)}")

# Initialize variables
train = True
max_trial_length = 0  # Track maximum trial length across sessions

# Iterate over all sessions
for file_path in trials:
    data_path = file_path / "data.npy"
    _labels_path = file_path / "labels.npy"

    # Ensure the files exist before processing
    if not data_path.exists() or not _labels_path.exists():
        print(f"Skipping session {file_path.name}: Missing data or labels file.")
        continue

    # Load labels
    _labels = np.load(_labels_path, allow_pickle=True)

    # Perform train/validation split
    if train:
        selection = np.concatenate(
            [np.argwhere(_labels == label_id).flatten()[:-5] for label_id in np.unique(_labels)]
        )
    else:
        selection = np.concatenate(
            [np.argwhere(_labels == label_id).flatten()[-5:] for label_id in np.unique(_labels)]
        )

    # Load selected data
    selected_data = np.load(data_path, allow_pickle=True)[selection]

    # Update max trial length
    session_max_length = selected_data.shape[2]  # Assuming shape is (n_trials, n_channels, n_samples)
    max_trial_length = max(max_trial_length, session_max_length)

    print(f"Session {file_path.name}: Max trial length = {session_max_length}")

# Print overall max trial length
print("Overall MAX TRIAL LENGTH:", max_trial_length)


Found sessions: 11
Session S01: Max trial length = 7196
Session S03: Max trial length = 7593
Session S04: Max trial length = 6545
Session S05: Max trial length = 7744
Session S06: Max trial length = 7062
Session S07: Max trial length = 6748
Session S08: Max trial length = 14997
Session S09: Max trial length = 6515
Session S10: Max trial length = 6161
Session S11: Max trial length = 6894
Session S12: Max trial length = 6479
Overall MAX TRIAL LENGTH: 14997


In [15]:
import pandas as pd
session = "S01"
marker = pd.read_csv(f"C:\\Users\\msi\\Desktop\\Constanze\\Docs\\DATA\\marker\\{session}.csv")
textmaps = marker.iloc[1:-1,0:2]

# Save the DataFrame as a JSON file
output_path = f"C:\\Users\\msi\\Desktop\\Constanze\\Docs\\DATA\\marker\\textmaps\\{session}.json"
textmaps.to_json(output_path, orient='records', lines=True)

print(f"Data saved as JSON at: {output_path}")

Data saved as JSON at: C:\Users\msi\Desktop\Constanze\Docs\DATA\marker\textmaps\S01.json


In [16]:
import json

# Load your JSON file
input_path = "C:\\Users\\msi\\Desktop\\Constanze\\Docs\\DATA\\marker\\textmaps\\S01.json"
output_path = "C:\\Users\\msi\\Desktop\\Constanze\\Docs\\DATA\\marker\\textmaps\\S01_transformed.json"

with open(input_path, "r") as file:
    data = [json.loads(line) for line in file]

# Transform into the desired format
transformed_data = {entry["sentences"]: int(entry["labels"]) for entry in data}

# Save the transformed data
with open(output_path, "w") as file:
    json.dump(transformed_data, file, ensure_ascii=False, indent=4)

print(f"Transformed JSON saved to: {output_path}")


Transformed JSON saved to: C:\Users\msi\Desktop\Constanze\Docs\DATA\marker\textmaps\S01_transformed.json
